In [1]:
import numpy as np
from skimage.io import imread, imsave
import glob
from skimage.morphology import skeletonize, opening, dilation, thin
import statistics as stats
import matplotlib.pyplot as plt
from skimage.measure import regionprops
from track.track import trackYeasts, track_obj
import numpy as np
from skimage.measure import regionprops
from skimage.morphology import thin, skeletonize, opening, dilation, erosion
import statistics
from track.utils import get_bbox_coords
import statistics as stats
from track.cell import getCellData
import matplotlib.pyplot as plt

In [2]:
im_paths = sorted(glob.glob("test_images/Pos2_MAT/*000.tif"))
cp_masks_paths = sorted(glob.glob("test_images/Pos2_MAT/*masks.tif"))
ims = [imread(p) for p in im_paths]
mating_masks = [imread(m) for m in cp_masks_paths]
artilife_paths = sorted(glob.glob("test_images/Pos2_ARTI/*cp_masks.tif"))
cp_masks = [imread(p) for p in artilife_paths][:101]
len(ims), len(mating_masks), len(cp_masks)

(101, 101, 101)

In [ ]:
do_skeletonize = True
masks = mating_masks

MATC = [[],[]]
for i in range(len(ims)-1):
    I1 = ims[i].copy()
    I2 = masks[i+1].copy()
    IS6 = np.zeros_like(I2)

    if i==0:
        ccel = 1
        I3=(I2==ccel)
        r_slice, c_slice = get_bbox_coords(I3)
        I3A = I3.copy()[r_slice, c_slice]
    else:
        I3A = MATC[0][i-1].copy()[r_slice, c_slice]
    
    if do_skeletonize:
        I3A = skeletonize((I3A>0).astype(np.uint8), method="zhang")
    else:
        r_centroid,c_centroid = regionprops((I3A>0).asytpe(np.uint8))[0].centroid
        r_centroid, c_centroid =int(r_centroid), int(c_centroid)
        I3A = np.zeros_like(I3A, dtype = np.uint8)
        I3A[r_centroid,c_centroid] = 1
        I3A = dilation(I3A, np.ones((3,3)))
    I2A = I2.copy()[r_slice, c_slice]

    if np.sum(I2A.astype(np.uint8))==0:
        I2A = MATC[0][i-1].copy()[r_slice, c_slice]

    
    I3B = np.multiply(I3A.astype(np.uint16),(I2A.astype(np.uint16)))
    ind = stats.mode(I3B[I3B!=0])
    pix = np.nonzero(I2A==ind)

    I2A[pix] = 1

    IS6[r_slice, c_slice] = I2A.copy()
    r_slice, c_slice = get_bbox_coords(IS6)

    I22 = np.zeros_like(I2)
    pix1 = np.nonzero(IS6!=0)
    I2[pix1] = 0

    pix2 = np.unique(I2)
    pix2 = pix2[1:]
    for ity in range(0, len(pix2)):
        pix4 = np.nonzero(I2==pix2[ity])
        I22[pix4] = ity+1

    MATC[1].append(I22)
    MATC[0].append(IS6)

plt.imshow(MATC[0][-1])
plt.show()


# loop 2
ATC = 1
while np.sum(ATC)!=0:
    A1 = np.zeros((1,len(ims)))
    A4 = 1
    A5 = 1
    #for i in range(len(ims)-2):
    for i in [0]:
        A2 = opening(MATC[1][i], footprint = np.ones((3,3)))

        if A2.sum() == 0 and A5!=0:
            A1[0,i]=0
            A5 = 1
        else:
            A1[0,i]=1
            A5 = 0

            if i==0:
                ccel=1
                I3 = MATC[1][i]==ccel
                r_slice, c_slice = get_bbox_coords(I3)
                I3A = I3[r_slice, c_slice]
            elif A1[0,i]==1 and A1[0, i-1]==0 and A4!=0:
                ccel = np.min(A2[A2!=0])
                I3 = MATC[1][i]==ccel
                r_slice, c_slice = get_bbox_coords(I3)
                I3A = I3[r_slice, c_slice]
                A4 = 0
            else:
                I3A = MATC[0][i-1].copy()[r_slice, c_slice]
                I3A[I3A!=np.max(I3A)] = 0
            
            I2 = MATC[1][i+1].copy()

            if skeletonize:
                I3A = skeletonize((I3A>0).astype(np.uint8), method="zhang")
            else:
                r_centroid,c_centroid = regionprops((I3A>0).asytpe(np.uint8))[0].centroid
                r_centroid, c_centroid =int(r_centroid), int(c_centroid)
                I3A = np.zeros_like(I3A, dtype = np.uint8)
                I3A[r_centroid,c_centroid] = 1
                I3A = dilation(I3A, np.ones((3,3)))

            I2A = I2.copy()[r_slice, c_slice]

            if I2A.sum()==0:
                I2A = MATC[0][i-1].copy()[r_slice, c_slice]
                #print("I2A shape", I2A.shape)
                pC1 = round(I2A.shape[0]/2)
                pC2 = round(I2A.shape[1]/2)

                I2AA = np.zeros_like(I2A)
                I2AA[pC1,pC2]=1
                I2AA = dilation(I2AA, footprint = np.ones((9,9)))
                I2AA1 = np.multiply(I2AA, I2A)
                pixR = np.nonzero(I2A!=np.max(I2AA1))
                I2A[pixR] = 0
            
            I3B = np.multiply(I3A.astype(np.uint16),I2A.astype(np.uint16))

            if I3B.sum()==0:
                I2A = MATC[0][i-1].copy()[r_slice, c_slice]
                pC1 = round(I2A.shape[0]/2)
                pC2 = round(I2A.shape[1]/2)

                I2AA = np.zeros_like(I2A)
                I2AA[pC1, pC2]=1
                I2AA = dilation(I2AA,footprint = np.ones((9,9)))
                I2AA1 = np.multiply(I2AA, I2A)
                pixR = np.nonzero(I2A!=np.max(I2AA1))
                I2A[pixR] = 0
            
            I3B = np.multiply(I3A.astype(np.uint16),I2A.astype(np.uint16))
            
            if I3B.sum()==0:
                I2A = MATC[1][i].copy()[r_slice, c_slice]
                pC1 = round(I2A.shape[0]/2)
                pC2 = round(I2A.shape[1]/2)

                I2AA = np.zeros_like(I2AA)
                I2AA[pC1, pC2]=1
                I2AA = dilation(I2AA,footprint = np.ones((9,9)))
                I2AA1 = np.multiply(I2AA, I2A)
                pixR = np.nonzero(I2A!=np.max(I2AA1))
                I2A[pixR] = 0
            
            I3B = np.multiply(I3A.astype(np.uint16),I2A.astype(np.uint16))

            if I3B.sum()==0:
                I2A = MATC[1][i].copy()[r_slice, c_slice] 
                I3A = skeletonize((I2A>0).astype(np.uint8), method = "zhang") 
                I3B = np.multiply(I3A.astype(np.uint16),I2A.astype(np.uint16))
            
            ind = stats.mode(I3B[I3B!=0])
            pix = np.nonzero(I2A==ind)
            pixE = np.nonzero(I2A!=ind)
            I2A[pixE]=0

            I2A[pix] = np.max(MATC[0][-2])+1

            IS6B = np.zeros_like(I2)
            IS6B[r_slice, c_slice] = I2A.copy()
            r_slice, c_slice = get_bbox_coords(IS6B>0)
            
            pixF = np.nonzero(IS6B!=0)
            IS6C = MATC[0][i].copy()
            IS6C[pixF] = np.max(MATC[0][-2])+1
            MATC[0][i] = IS6C.copy()

            I2D = MATC[1][i]
            I2E = (IS6C>0).astype(np.uint8)
            plt.imshow(I2E)
            plt.show()
            I2D1 = np.multiply(I2D, I2E)
            I2D1 = erosion(I2D1, footprint = np.ones((3,3)))
            pix2 = np.nonzero(I2D1!=0)
            cell_E = stats.mode(I2D[pix2])
            I2D[I2D==cell_E] = 0

            I22 = np.zeros_like(I2D)

            pix2 = np.unique(I2D)
            pix2 = pix2[1:]
            for ity in range(0,len(pix2)):
                BBB = erosion((I2D == pix2[ity]).astype(np.uint8), 
                            footprint = np.ones((3,3)))

                if np.mean(BBB) > 0:
                    pix4 = np.nonzero(I2D==pix2[ity])
                    I22[pix4] = ity
                
            
            MATC[1][i]=I22.copy()

        ATC = np.unique(MATC[1][i])
Matmasks = [0]*(len(ims)-2)
for i in range(len(ims)-2):
    Matmasks[i] = MATC[0][i]